In [ ]:
#import the required libraries 

import win32com.client as client      # win32com 
from datetime import datetime
from bs4 import BeautifulSoap
import pandas as pd
import re


outlook=client.Dispatch("Outlook.application")   # Dispatch method interacts with IDispatch-based COM object, in our case it is Outlook Application
namespace=outlook.GetNamespace("MAPI")  # We Use GetNameSpace ("MAPI") to return the Outlook NameSpace object from the Application object


for folder in namespace.Folders:  # To print all the folders and get the list of all available folders
        print(folder.Name)
    
psd2=namespace.Folders["folder_name_1"] # Select the desired folder using "Folder" property of GetNamespace object. 
print(psd2.Name)
sub=psd2.Folders["sub_folder_name"]              # Select sub folder
print(sub)

'''
#----------Extras--------------#
sub=psd2.Folders["Splunk Alerts"]
print(sub)
content=sub.Items(1).body                   #We can also use index to extract the content of a folder from email
f=open("msg.txt","w")
f.writelines(content)
f.close()

d1,d2="2021-12-09T14:00:00", "2021-12-09T16:00:00"
dd1=datetime.strptime(d1, '%Y-%m-%dT%H:%M:%S')  #strptime means string parser, this will convert 
dd2=datetime.strptime(d2, '%Y-%m-%dT%H:%M:%S')  #a string format to datetime.
print(type(dd1))
dt1=datetime.strftime(dd1, '%m/%d/%Y %I:%M:%S %p')  #strftime means string formatter, this will format 
dt2=datetime.strftime(dd2, '%m/%d/%Y %I:%M:%S %p')  #a datetime object to string format. %p used for AM/PM
print(type(dt1))
'''

#To manually set the time in order to filter the mails .
# Use the "Items" property to return the Items object of a Folder object.
# Restrict method used with "ReceivedTime" property of MailItem object to set the time frame in order to filter the mails

message=sub.Items.Restrict("[ReceivedTime] >='27/12/2021 04:00 AM' AND [ReceivedTime] >='27/12/2021 04:00 AM' ")

#print(type(messages))
#print(messages)

#Use regular expression to match required data from mail body

Event_pattern=r"\bpattern=([^,]+),"     # '\b' matches the empty string at the beginning or end of a word.. [^,]+ searches for everything except a comma
Error_pattern=r"pattern1\"\:.*|pattern2\:.*|pattern3\:.*"  # .* matches 0 or more characters
#print(Event_pattern,Error_pattern)


 #Iterating over the message body and extract all the required data
    
for i in messages :            
    subject=i.Subject           #"Subject" is MailItem object's property
    subj=subject[14:]           #Extract only required subject text
    dt=i.ReceivedTime.strftime('%d-%m-%y')
    tt=i.ReceivedTime.strftime('%I:%M %p') #To get time in 12 hrs and we use %p for AM/PM
    
    date={'Date':[dt],'Time':[tt]}        #create didctionary to insert Date and time into below Dataframe
    temp_df=pd.DataFrame(data,index=None) #Initial DF created with above values .
    
    if subj=="Subject_1":  #Based on subject line add required data as columns to temp_df dataframe
        temp_df.insert(2,'Error',subj)      #Insert the data to desired column using the "insert()"
        temp_df.insert(3,'Event Type'," ")
        temp_df.insert(4,'Pod/Container'," ")
        temp_df.insert(5,'Error Body',"There may be issue with EIDP")
        temp_df.insert(6,'Xreq-Id',"")
        temp_df.insert(7,'Xreq-Id Count',0)
        temp_df.insert(8,'Remarks'," ")
        temp_df.insert(9,'Action Taken',"Captured in sheet")
        
        with open("splunk.csv",'a') as fp:  #writing our dataframe to a already saved csv file with mode='a' ,index=False,header=False
            temp_df.to_csv(fp,mode='a', index=False, header=False)
    
    else:
        htmlbody=i.HTMLbody                                         #Extract HTML body from message , HTMLbody is property of MailItem
        msg_df=pd.read_html(htmlbody,header=0,index_col=None)[0]    # To Read HTML tables into a list of DataFrame objects and get table at 0th position
        
        #msg_df.to_csv("abc.csv",mode='a',index=False)
        #print(msg_df.head())
        
        if subj=="Subject_2":  #Based on subject line add required data as columns to temp_df
        
            Error=msg_df['errorMsg'].tolist()  #convert the column into a list, 'errorMsg' is a column name from mail body 
            for j in Error:
                if isinstance(j,str):          #to check if anything else present apart from string data like space " ",or special character.
                    continue
                else:
                    Error=map(str,Error)       #if other than string, convert all data to str and break. Map returns a List
                    break
            poddata=msg_df['pod'].tolist()
            XreqId=msg_df['Request_ID'].tolist()
            
            temp_df.insert(2,'Error',subj)
            temp_df.insert(3,'Event Type',("\n".join(set(re.findall(Event_pattern,"\n".join(Error)))))) #First match the pattern in Error msg body by
            temp_df.insert(4,'Pod/Container',("\n".join(set(poddata))))                                 #converting it to string, then copy only unique
            temp_df.insert(5,'Error Body',("\n".join(set(re.findall(Error_pattern,"\n".join(Error)))))) # value in "Error body" or "Event Type"  
            temp_df.insert(6,'Xreq-Id',("\n".join(XreqId)))
            temp_df.insert(7,'Xreq-Id Count',len(XreqId))
            temp_df.insert(8,'Remarks'," ")
            temp_df.insert(9,'Action Taken',"Captured in sheet")
            
            with open("splunk.csv",'a') as fp:     
                temp_df.to_csv(fp,mode='a', index=False, header=False)    
            
        elif subj=="Subject_3":  #Based on subject line add required data as columns to temp_df
        
            Error=msg_df['errorMsg'].tolist()  #convert the column into a list, 'errorMsg' is from mail body a column name
            for j in Error:
                if isinstance(j,str):          #to check if anything else present apart from string data like space " ",or special character.
                    continue
                else:
                    Error=map(str,Error)       #if other than string, convert all data to str and break. Map returns a List
                    break
            poddata=msg_df['pod'].tolist()
            XreqId=msg_df['Request_ID'].tolist()
            
            temp_df.insert(2,'Error',subj)
            temp_df.insert(3,'Event Type',("\n".join(set(re.findall(Event_pattern,"\n".join(Error)))))) #First match the pattern in Error msg body by
            temp_df.insert(4,'Pod/Container'," ")                                                       #converting it to string, then copy only unique
            temp_df.insert(5,'Error Body',("\n".join(set(re.findall(Error_pattern,"\n".join(Error)))))) # value in "Error body" or "Event Type"  
            temp_df.insert(6,'Xreq-Id',("\n".join(XreqId)))
            temp_df.insert(7,'Xreq-Id Count',len(XreqId))
            temp_df.insert(8,'Remarks',"NA")
            temp_df.insert(9,'Action Taken',"Captured in sheet")
            
            with open("splunk.csv",'a') as fp:      #writing to a already saved csv file with mode='a' ,index=False,header=False
                temp_df.to_csv(fp,mode='a', index=False, header=False)                
                
        
            
#-------To fetch mail from Geneos folders

psd2=namespace.Folders["folder_name_2"] # Select the desired folder and subfolder   
print(psd2.Name)
sub=psd2.Folders["sub_folder_name"]              # Select sub folder
print(sub)        

#To manually set the time in order to filter the mails .
messages=sub.Items.Restrict("[ReceivedTime] >='27/12/2021 04:00 AM' AND [ReceivedTime] >='27/12/2021 04:00 AM' ")

for i in messages:
    htmlbody=i.HTMLbody
    #print(htmlbody)
    
    Bsoup=BeautifulSoap(htmlbody,'html.parser')
    print(Bsoup.b.text)
    Error=Bsoup.b.text #Extracting the <b> tag text , hence used beautiful Soap as I was not able to extract using pandas 
    
    #with open ('geneos_html.txt",'w') as gh:
        #gh.writelines(htmlbody)
        
    subject=i.Subject
    subj=subject[0:] #fetch full subject line
    dt=i.ReceivedTime.strftime('%d-%m-%y')
    tt=i.ReceivedTime.strftime('%I:%M %p') #To get time in 12 hrs and we use %p for AM/PM
    
    date={'Date':[dt],'Time':[tt]}        #create didctionary to insert into below Dataframe
    temp_df=pd.DataFrame(data,index=None) #Initial DF created with above values .
    
    temp_df.insert(2,'Error',subj)                  
    temp_df.insert(3,'Pod/Container',"PSD2-CE PROD ")
    temp_df.insert(4,'Error Description',Error)
    temp_df.insert(5,'Remarks'," ")
    temp_df.insert(6,'Action Taken',"Captured in sheet")
    
    with open("geneos.csv",'a') as fp:
            temp_df.to_csv(fp,mode='a', index=False, header=False)
            

#----------Extras--------------
'''
subject  =sub.Items.GetLast().Subject
subj=subject[14:]
dt=sub.Items.GetLast().SentOn.strftime('%d-%m-%y') #to fetch the latest mail from folder. We can user GetFirst() for oldest Mail
tt=sub.Items.GetLast().SentOn.strftime('%d-%m-%y')          
print(dt,tt)

content=sub.Items.GetLast().HTMLbody
df=pd.read_html(content,header=0,index_col=None)[0]
df.reset_index(drop=True,inplace=True)
data={'Data':[dt],'Time':[tt],'Error':[subj]}
dff=pd.DataFrame(data,index=None)
poddata=df['pod'].tolist()
dff.insert(3,'pod/container',("\n".join(poddata)))
'''